In [2]:
from pycocotools.coco import COCO
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import pandas as pd
import yaml
%matplotlib inline

In [3]:
coco = COCO("../object_detection/train.json")
coco_eval = COCO("../object_detection/eval.json")

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '../object_detection/train.json'

In [ ]:
cat_df = pd.DataFrame.from_dict(coco.cats, orient='index')
ann_df = pd.DataFrame.from_dict(coco.anns, orient='index')
train_img_df = pd.DataFrame.from_dict(coco.imgs, orient='index')
eval_img_df = pd.DataFrame.from_dict(coco_eval.imgs, orient='index')

In [ ]:
cat_df

In [ ]:
def images_eda(image_df):
    n_total = len(image_df)
    dims = pd.Series(tuple(zip(image_df.width, image_df.height)))
    print(f'Total number of images in dataset: {n_total}')
    print('Top resolutions:')
    print(dims.value_counts().head(25))

# images_eda(train_img_df)

In [ ]:
def object_eda(annotation_df, categories_df):
    cat_count = annotation_df.category_id.value_counts().to_frame()
    df = categories_df.join(cat_count).sort_values('category_id', ascending=False).reset_index(drop=True)
    df = df.rename(columns={'id': 'category_id',
                              'name': 'category',
                              'supercategory': 'supercategory',
                              'category_id': 'cat_id_counts'})
    # df = df[df.cat_id_counts > 50]
    print(f'There are {len(df.dropna())} of 290 species present in the dataset.')
    print(f'There are {len(df.dropna().supercategory.unique())} of 20 semantic supercategories present.')
    return df

df = object_eda(ann_df, cat_df).dropna()
df

In [ ]:
super_dict = {}
for i, supercat in enumerate(df.supercategory.unique()):
    df2 = df[df.supercategory == supercat]
    nsup = sum(df2['cat_id_counts'])
    
    most_common_loc =df2['cat_id_counts'].max()
    # print(supercat, nsup, most_common_loc)
    most_common = int(df2[df2.cat_id_counts == most_common_loc]['category_id'])
    super_dict[i] = {'supercat_id': i,
                     'supercategory': supercat,
                     'top_category_id': most_common,
                     'cat_id_percentage': most_common_loc / nsup}

df_super = pd.DataFrame.from_dict(super_dict, orient='index')
df_super

In [ ]:
supercat_map = df_super[['supercat_id', 'supercategory']].to_dict()['supercategory']

In [ ]:
dataset = {'path': '/data/dataset',
           'train': 'images/train',
           'val': 'images/val',
           'names': supercat_map}

In [ ]:
dataset

In [ ]:
with open("/data/super_dataset.yaml", 'w') as yamlfile:
    data = yaml.dump(dataset, yamlfile)
    print("Write successful")

In [ ]:
df_supertop = df.merge(df_super, on='supercategory')

In [ ]:
# df_super.to_json('../supercat_key.json')

In [ ]:
ann_df

In [ ]:
ann_super_df = ann_df.merge(df_supertop, on='category_id').sort_values('id').reset_index(drop=True)

In [ ]:
ann_super_df.head(1)

In [ ]:
ann_out = ann_super_df[['id', 'image_id', 'category_id', 'segmentation', 'area', 'bbox', 'iscrowd']]
ann_out = ann_out.assign(category_id=ann_super_df['supercat_id'])
ann_out

In [ ]:
# ann_out.to_json('../annotation_sup.json')

In [ ]:
# ann_super_df.to_json('../master_key.json')

In [ ]:
# super_map = {}
# for i, cat in enumerate(df.category_id.unique()):
#     df2 = df[df.category_id == cat]
#     # print(supercat, df[df.supercategory == supercat]['category_id'].unique())
#     super_map[cat] = df2.supercategory.values
# super_map

In [ ]:
def supercat_eda(annotation_df, categories_df):
    df = categories_df
    out = {}
    supermap = {i: list(df[df.supercategory == i].id.unique()) for i in df.supercategory.unique()}
    for i in supermap.keys():
        out[i] = 0
        for j in supermap[i]:
            out[i] += (len(annotation_df[annotation_df.category_id == j].image_id.unique()))    
    return out, supermap

supercat_eda(ann_df, cat_df)#.head(15)